<a href="https://colab.research.google.com/github/KevinChenWu/IE-0624-Laboratorio-5/blob/main/src/Laboratorio5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 5

Estudiante: Kevin Chen Wu<br/>
Carné: B92215<br/><br/>
Este es el Jupyter Notebook creado en Google Colab para realizar la construcción y entrenamiento del modelo HAR (Human Activity Recognition, en inglés) mediante TensorFlow para la placa Arduino Nano 33 BLE Sense.

## Setup del ambiente de ejecución (Python)

Se instala las bibliotecas y dependencias necesarias para el notebook, es necesario ejecutarlo.

In [1]:
!apt-get -qq install xxd
!pip install pandas numpy
!pip install tensorflow==2.11.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Datos para construcción y entrenamiento del modelo

Se debe subir los archivos csv del registro de los movimientos a la carpeta "/content".

## Entrenamiento de la Red neuronal

### Preparación de datos
Se toma los datos de los archivos csv subidos y se convierten a un dataframe de pandas para entrenar la red neuronal.

Se debe actualizar la lista "Gesture" con los nombres de los archivos csv.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

print(f"TensorFlow version = {tf.__version__}\n")

# Se fija una semilla random para obtener los mismo números randoms
# en cada ejecución de este notebook
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Lista de gestos
GESTURES = [
    "punch",
    "like",
    "circle"
]

SAMPLES_PER_GESTURE = 32

NUM_GESTURES = len(GESTURES)

# Se crea un matriz codificado en One-Hot para usarse en la salida
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

# Se lee cada archivo csv file and y crea una entrada y salida
for gesture_index in range(NUM_GESTURES):
  gesture = GESTURES[gesture_index]
  print(f"Processing index {gesture_index} for gesture '{gesture}'.")
  
  output = ONE_HOT_ENCODED_GESTURES[gesture_index]
  
  df = pd.read_csv("/content/" + gesture + ".csv")
  
  # Se calcula el número de gestos guardados en cada archivo
  num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)
  
  print(f"\tThere are {num_recordings} recordings of the {gesture} gesture.")
  
  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i * SAMPLES_PER_GESTURE + j
      # Se normaliza los datos de entrada de 0 a 1:
      # - La aceleracción está entre -4 a +4
      # - El giroscopio está entre -2000 a +2000
      tensor += [
          (df['gX'][index] + 2000) / 4000,
          (df['gY'][index] + 2000) / 4000,
          (df['gZ'][index] + 2000) / 4000
      ]

    inputs.append(tensor)
    outputs.append(output)

# Se convierte la lista a un arreglo de numpy
inputs = np.array(inputs)
outputs = np.array(outputs)

print("Data set parsing and preparation complete.")

TensorFlow version = 2.11.0

Processing index 0 for gesture 'punch'.
	There are 32 recordings of the punch gesture.
Processing index 1 for gesture 'like'.
	There are 32 recordings of the like gesture.
Processing index 2 for gesture 'circle'.
	There are 32 recordings of the circle gesture.
Data set parsing and preparation complete.


### Aleatorización y división de pares de entradas y salidas para entrenamiento

Aleatoriamente se divide los pares de entradas y salidas en conjuntos de datos: 60% para entrenamiento, 20% para validación y 20% para pruebas.

In [3]:
# Se aleatoriza el orden de las entradas para distribuirlos equitativamente
# en entrenamiento, validación y pruebas
# https://stackoverflow.com/a/37710486/2020087
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# Se intercambia los índices consecutivos (0, 1, 2, etc) con índices randoms
inputs = inputs[randomize]
outputs = outputs[randomize]

# Se divide los datos en 3 conjuntos: entrenamiento, validación y pruebas
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, \
inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])

outputs_train, outputs_test, \
outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Data set randomization and splitting complete.")

Data set randomization and splitting complete.


### Construcción y entrenamiento del modelo

Se construye y se entrena un modelo [TensorFlow](https://www.tensorflow.org) usando API de alto nivel [Keras](https://www.tensorflow.org/guide/keras).

In [4]:
# Se construye el modelo y se entrena
model = tf.keras.Sequential()
# Se usa ReLu como función de activación
model.add(tf.keras.layers.Dense(50, activation="relu"))
model.add(tf.keras.layers.Dense(15, activation="relu"))
# Se usa softmax para la capa de salida (se asume un gesto a la vez)
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax'))
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(
    inputs_train, outputs_train, 
    epochs=600, batch_size=1, 
    validation_data=(inputs_validate, outputs_validate)
)

Epoch 1/600
57/57 [==============================] - 2s 8ms/step - loss: 0.2252 - mae: 0.4447 - val_loss: 0.2396 - val_mae: 0.4587
Epoch 2/600
57/57 [==============================] - 0s 3ms/step - loss: 0.2242 - mae: 0.4430 - val_loss: 0.2375 - val_mae: 0.4565
Epoch 3/600
57/57 [==============================] - 0s 4ms/step - loss: 0.2245 - mae: 0.4425 - val_loss: 0.2268 - val_mae: 0.4485
Epoch 4/600
57/57 [==============================] - 0s 4ms/step - loss: 0.2234 - mae: 0.4446 - val_loss: 0.2292 - val_mae: 0.4507
Epoch 5/600
57/57 [==============================] - 0s 4ms/step - loss: 0.2254 - mae: 0.4442 - val_loss: 0.2410 - val_mae: 0.4601
Epoch 6/600
57/57 [==============================] - 0s 4ms/step - loss: 0.2242 - mae: 0.4441 - val_loss: 0.2474 - val_mae: 0.4641
Epoch 7/600
57/57 [==============================] - 0s 4ms/step - loss: 0.2237 - mae: 0.4412 - val_loss: 0.2345 - val_mae: 0.4546
Epoch 8/600
57/57 [==============================] - 0s 4ms/step - loss: 0.2218 - m

## Conversión del modelo entrenado a Tensor Flow Lite

Se convierte el modela al formato TFlite. Se indica también el tamaño del modelo en bytes.

In [5]:
# Se convierte el modelo al formato TensorFlow Lite sin cuantización
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Se salva el modelo al disco
open("gesture_model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Model is %d bytes" % basic_model_size)

Model is 24744 bytes


### Codificación del modelo en un archivo header de Arduino

Se convierte el modelo en un arreglo constante de bytes que contiene el modelo TFlite.

In [6]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.\n")
print("Open the side panel (refresh if needed).\n")
print("Double click model.h to download the file.")

Header file, model.h, is 152,622 bytes.

Open the side panel (refresh if needed).

Double click model.h to download the file.
